#   LangChain Tools

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### 1. **SQLDatabaseToolkit**

- **SQLDatabaseToolkit**은 LLM이 **데이터베이스와 상호작용**할 수 있게 해주는 특화 도구

- 자연어를 **SQL 쿼리로 변환**하고 데이터베이스 작업을 수행하는 기능 제공

- **테이블 스키마 분석**, **쿼리 실행**, **결과 해석** 등 DB 관련 작업을 자동화

`(1) SQLDatabaseToolkit 초기화`

In [ ]:
# 데이터베이스 검색 (내장 도구)
from langchain_community.agent_toolkits import SQLDatabaseToolkit 
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

# 데이터베이스 연결 및 도구 생성 
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
db = SQLDatabase.from_uri("sqlite:///etf_database.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# 도구 목록 출력
toolkit.get_tools()

`(2) QuerySQLDatabaseTool`

- **QuerySQLDatabaseTool**은 SQL 쿼리를 실행하고 데이터베이스 결과를 반환하는 도구
- 잘못된 쿼리 입력 시 **오류 메시지**를 통해 문제점 확인 가능
- **sql_db_schema** 도구와 연계하여 테이블 구조 확인 및 쿼리 수정 가능

In [ ]:
from langchain_community.tools import QuerySQLDatabaseTool

# 데이터베이스 쿼리 도구 생성
query_excecuter = QuerySQLDatabaseTool(db=db)

# 데이터베이스 쿼리 실행
query = "SELECT count(*) FROM ETFs;"
result = query_excecuter.invoke(query)

# 결과 출력
print(result)

`(3) ListSQLDatabaseTool`

- **ListSQLDatabaseTool**은 데이터베이스의 **전체 테이블 목록**을 조회하는 도구
- 다른 SQL 도구 사용 전 **테이블 존재 여부** 확인에 활용

In [ ]:
from langchain_community.tools import ListSQLDatabaseTool

# 데이터베이스 리스트 도구 생성
list_excecuter = ListSQLDatabaseTool(db=db)

# 데이터베이스 리스트 실행
tables = list_excecuter.invoke("")

# 결과 출력
print(tables)

`(4) InfoSQLDatabaseTool`

- **InfoSQLDatabaseTool**은 테이블의 **스키마 정보**와 **샘플 데이터**를 제공하는 도구
- **ListSQLDatabaseTool**을 먼저 실행하여 사용 가능한 테이블 목록 확인 필요
- 여러 테이블 정보를 한 번에 조회할 때는 **쉼표로 구분**하여 입력

In [ ]:
from langchain_community.tools import InfoSQLDatabaseTool

# 데이터베이스 정보 도구 생성
info_excecuter = InfoSQLDatabaseTool(db=db)

# 데이터베이스 정보 실행
result = info_excecuter.invoke("ETFs")

# 결과 출력
print(result)

`(5) QuerySQLCheckerTool`

- **QuerySQLCheckerTool**은 SQL 쿼리의 **오류를 자동 검사**하는 도구
- 문법적/논리적 오류를 찾아내고 **수정된 쿼리**를 제공
- 쿼리 실행 전 **사전 검증**으로 오류 발생 가능성 최소화

In [ ]:
from langchain_community.tools import QuerySQLCheckerTool

# 데이터베이스 쿼리 검사 도구 생성
query_checker = QuerySQLCheckerTool(llm=llm, db=db)

# 데이터베이스 쿼리 검사 실행
query = "SELECT * FROM ETFs count 3;"
result = query_checker.invoke(query)

# 결과 출력
print(result)


`(6) SQL 에이전트 생성 및 실행`

> - **`create_agent`** 사용
> - `system_prompt` 매개변수 사용

In [ ]:
# Hub에서 프롬프트 템플릿 로드
from langchain_classic import hub
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

# 시스템 메시지 생성
system_message = prompt_template.format(dialect="SQLite", top_k=5)

# 시스템 메시지 확인
pprint(system_message)

In [ ]:
from langchain.agents import create_agent

# SQL 에이전트 생성 
agent = create_agent(
    model=llm,
    tools=toolkit.get_tools(),
    system_prompt=system_message
)

In [ ]:
# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content": "ETF 종목은 모두 몇 개인가요?"}]},
)

# 에이전트 실행 결과 출력
pprint(response['messages'])

In [ ]:
for msg in response['messages']:
    msg.pretty_print()

---
### **[실습]**

- 시스템 프롬프트 내용을 확인하고, 개선할 포인트를 찾아서 적용합니다. 
- 개선된 프롬프트를 적용해서 SQL 쿼리 에이전트를 정의합니다. 
- 데이터베이스 내용을 쿼리하는 질문을 테스트합니다. 

In [ ]:
# 여기에 코드를 작성하세요.

---

### 2. **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `uv add langchain-tavily` 또는 `pip install langchain-tavily`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [ ]:
from langchain_tavily import TavilySearch

# Tavily 검색 도구 생성
tavily_search = TavilySearch(
    max_results=5, 
    topic="general",  # news, finance, general
)

`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [ ]:
result = tavily_search.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [ ]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-nano")

# 모델에 도구 등록
model_with_tools = model.bind_tools([tavily_search])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

In [ ]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = tavily_search.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

In [ ]:
json.loads(tool_msg.content)['results']

`(4) Agent 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 에이전트 실행

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage
from langchain.agents import create_agent
import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-nano")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
system_message =  f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."

# 에이전트 생성 
agent = create_agent(
    model=llm,
    tools=[tavily_search],
    system_prompt=system_message
)

# 에이전트 호출
result = agent.invoke(
    {"messages": [("user", "한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")]},
)

for msg in result['messages']:
    msg.pretty_print()

---

### 3. **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구 정의`

In [ ]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) Agent 연계`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage
from langchain.agents import create_agent
import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-nano")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
system_message =  f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."

# 에이전트 생성 
agent = create_agent(
    model=llm,
    tools=[naver_news_search],
    system_prompt=system_message
)

# 에이전트 호출
result = agent.invoke(
    {"messages": [("user", "최근 주목받고 있는 ETF는 무엇인가요?")]},
)

for msg in result['messages']:
    msg.pretty_print()